In [ ]:
# %pip install tensorflow==2.6.2

In [ ]:
# Importing libraries

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import numpy as np
import scipy.io
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import keras.backend as K
import numpy as np
import os
import random
import seaborn as sns
from datetime import datetime
from random import randrange
import matplotlib.pyplot as plt
import math
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, \
    multiply, concatenate, Flatten, Activation, dot
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping
from IPython.display import FileLink, FileLinks
import warnings
import pprint
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
np.random.seed(42)

In [ ]:
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Data loading
mat = scipy.io.loadmat('/home/nicolo/Scrivania/mat/pydata.mat')
data = mat["data"]

# Data correction
temp = abs(data) > 3
data[temp] = data[temp]/1000
data.shape

In [ ]:
# Data preprocessing
psm1 = data[:,[1,2,3]].astype('float64')
psm2 = data[:,[4,5,6]].astype('float64')

cam = data[:,0].astype('int')
psm_features = data[:,[1,2,3,4,5,6]].astype('float64')

# psm1x = data[:,1].astype('float64')
# psm1y = data[:,2].astype('float64')
# psm1z = data[:,3].astype('float64')
# psm2x = data[:,4].astype('float64')
# psm2y = data[:,5].astype('float64')
# psm2z = data[:,6].astype('float64')

ecmx = data[:,7].astype('float64')
ecmy = data[:,8].astype('float64')
ecmz = data[:,9].astype('float64')

dist = np.sqrt(np.sum((psm1-psm2)**2, axis=1))

In [ ]:
# Cam = 2 signal elimination
loc = cam==2
cam[loc] = 0
np.sum(cam)

In [ ]:
# Clutch vector creation
difference = cam[1:]-cam[:-1]

press = np.where(difference==1)[0][:]
release_temp = np.where(difference==-1)[0][:]
release = release_temp[1:]

clutch_pressed = difference == 1
clutch_released = difference == -1

clutch = np.zeros(cam.shape[0]-1)
clutch[clutch_pressed] = 1
clutch = np.insert(clutch, 0, 0, axis=0)

print("Camera pedal has been pressed n times: ", press.shape[0])
print("Camera pedal has been released n times: ", release.shape[0])
print("First press at: ", press[0])
print("First release at: ", release[0])

In [ ]:
# Selection of usable windows
len=30
n_variables = 7
telescope = 3
samples = cam.shape[0]

p = press[1:]
r = release[:-1]

usable = np.where(clutch == 1)[0][:]
unusable = np.where((p-r)<len+telescope)[0][:]
clutch[press[unusable]+1] = 0
print("Usable clutches: ", np.sum(clutch))

In [ ]:
# Empty matrices creation
distance=np.expand_dims(dist, axis=1)

windows = np.zeros((cam.shape[0]-len-telescope,len,n_variables))
features = np.concatenate((psm_features,distance), axis=1)
print("Windows matrix dimensions: ", windows.shape)
print("Features matrix dimensions: ", features.shape)

label_vector = np.zeros((samples-len-telescope,1))
label_matrix_temp = np.zeros((samples-len-telescope,n_variables))

# If the starting clutch is before len+telescope, we cannot label it
index = np.where(clutch==1)
print("First camera movement at timestamp: ", index[0][0])

In [ ]:
# Useful matrices creation
label_vector[usable-len-telescope] = 1;
print('Matrix containing camera windows labels at each timestamp: ', label_vector.shape)
# label_matrix_temp[usable-len-telescope] = 1;

for j in range(n_variables):
    for i in range(samples-len-telescope):
        temp = features[i:i+len,j]
        windows[i,:,j] = temp

print("Matrix containing kinematic features at each timestamp: ", windows.shape)

In [ ]:
# Stratified train-test split
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(windows, label_vector, test_size=0.20, stratify=label_vector)

print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

In [ ]:
# Shapes
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
print('Input shape: ', input_shape)
print('Output shape: ', output_shape)

In [ ]:
import numpy as np
import math

# labels_dict : {ind_label: count_label}
# mu : parameter to tune 

def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weights = dict()
    
    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weights[key] = score if score > 1.0 else 1.0
    
    return class_weights

labels_dict = {0.0: 47107,
               1.0: 112}

class_weights = create_class_weight(labels_dict)
pprint.pprint(class_weights)

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0
    ):
    inputs = tfk.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in range(mlp_units):
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    outputs = layers.Dense(2, activation="softmax")(x)
    return tfk.Model(inputs,outputs)

In [ ]:
model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=128,
    mlp_dropout=0.4,
    dropout=0.25
    )

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tfk.optimizers.Adam(lr = 1e-4),
    metrics=["sparse_categorical_accuracy"]
)

model.summary()

callbacks = [tfk.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
             tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5,
                                        factor=0.1, min_lr=1e-7, verbose = 1)
]

history = model.fit(
    X_train,
    y_train,
    batch_size = 256,
    epochs = 200,
    validation_split=0.2,
    class_weight = class_weights,
    callbacks = callbacks
).history

model.evaluate(X_test, y_test, verbose=1)

In [ ]:
# df2 = df2[["correlation_Pwave80_lead1", "lag_Pwave80_lead1",
#        "lag_PRintervall80_lead1", "msd_PRintervall80_lead1",
#        "lag_beat80_lead1", "correlation_Pwave20_lead1", "lag_QRSwave20_lead1",
#        "lag_Pwave4_lead1", "lag_QRSwave4_lead1",
#        "correlation_PRintervall4_lead1", "lag_PRintervall4_lead1",
#        "values_under_mean_perc_Pwave_lead1", "mean_QRScomplex_lead1",
#        "median_QRScomplex_lead1", "values_over_mean_perc_QRScomplex_lead1",
#        "max_PRintervall_lead1", "min_PRintervall_lead1",
#        "kurtosis_PRintervall_lead1",
#        "values_under_mean_perc_PRintervall_lead1", "max_beat_lead1",
#        "min_beat_lead1", "values_under_mean_perc_beat_lead1",
#        "correlation_Pwave80_lead2", "lag_Pwave80_lead2",
#        "lag_PRintervall80_lead2", "lag_beat80_lead2",
#        "correlation_Pwave20_lead2", "lag_QRSwave20_lead2", "msd_beat20_lead2",
#        "lag_Pwave4_lead2", "lag_QRSwave4_lead2",
#        "correlation_PRintervall4_lead2", "lag_PRintervall4_lead2",
#        "lag_beat4_lead2", "values_under_mean_perc_Pwave_lead2",
#        "mean_QRScomplex_lead2", "median_QRScomplex_lead2",
#        "der_std_QRScomplex_lead2", "max_PRintervall_lead2",
#        "min_PRintervall_lead2", "kurtosis_PRintervall_lead2",
#        "values_under_mean_perc_PRintervall_lead2",
#        "max_min_diff_PRintervall_lead2", "max_beat_lead2", "min_beat_lead2",
#        "values_under_mean_perc_beat_lead2", "mean_rr_30s_back",
#        "std_drr_30s_back", "std_drr_60s_back", "mean_rr_120s_current",
#        "pNN50_120s_current", "rmssd_120s_current","beats","frequency"]]

In [ ]:
# df_dict = {}
# for i in range(320):
#     df_dict[f"df_predictions_{i}"] = pd.DataFrame(columns = ["predictions","predictions_letter"])
    
# for i in range(320):
#     matrix = df2["beats" == i]
#     if matrix["frequency"][1] == 250
#         prediction = model1.predict(X1)
#     else
#         prediction = model2.predict(X2)
    
#     df_dict[f"df_predictions_{i}"]["predictions"] = predictions
#     df_dict[f"df_predictions_{i}"]["predictions_letter"] = 'N' if df_dict[f"df_predictions_{i}"]["predictions_letter"] == 0 else  'S' if f_dict[f"df_predictions_{i}"]["predictions_letter"] == 1 else 'V'
    
# #     DF250.to_csv("tabella250.csv", index=False)
# #     DF128.to_csv("Satisfaction.csv", index=False)

In [ ]:
# DF250 = pd.DataFrame(prediction250)
# DF128 = pd.DataFrame(prediction128)
# DF250.to_csv("tabella250.csv", index=False)
# DF128.to_csv("Satisfaction.csv", index=False)